# Optimizing DNNs

This is companion code the Lecture 11, and close follows Chapter 11 of Geron. 

In this lab, we're going to show how to implement all of the DNN optimizations we discussed in Lecture 11 in Keras. This includes:

* __Selecting Activation Functions__: SELU, ELU, Leaky ReLU, ReLu. 
* __Choosing Weight Initializations__: He for ReLU variants, LeCun for SELU and Glorot for others. 
* __Batch Normalization__
* __Gradient Clipping__
* __Choosing Optimizers__: Momentum, Nesterov, AdaGrad, RMSProp, Adam, Nadam and AdaMax. 
* __Regularization__: $\ell_p$ regularization, Max-Norm, dropout and early stopping. 

### Note: 

In the notebook below I will be raining each model for 10 steps for time reasons. This is really not enough to see the differences between some of the gains as the initial model is still clearly making steady improvement at this stage. You should really train the model for at least 100 epochs. 

We'll start with the fashion MNIST dataset and a simple network:

In [6]:
import tensorflow as tf
from tensorflow import keras

fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]


model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_10 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_11 (Dense)             (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [7]:
## Compile the model with loss: sparse_categorical_crossentropy, optimizer: sgg, and metric: accuracy. 
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd",
             metrics=["accuracy"])

history = model.fit(X_train,y_train, epochs=10, 
          validation_data=[X_valid,y_valid])

## Fit the model for 10 epochs, vailidating on the validation data. 

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 9s 155us/sample - loss: 0.7207 - accuracy: 0.7617 - val_loss: 0.5234 - val_accuracy: 0.8146
Epoch 2/10
55000/55000 [==============================] - 8s 145us/sample - loss: 0.4893 - accuracy: 0.8302 - val_loss: 0.4527 - val_accuracy: 0.8470
Epoch 3/10
55000/55000 [==============================] - 6s 115us/sample - loss: 0.4428 - accuracy: 0.8463 - val_loss: 0.4313 - val_accuracy: 0.8532
Epoch 4/10
55000/55000 [==============================] - 7s 124us/sample - loss: 0.4163 - accuracy: 0.8546 - val_loss: 0.4230 - val_accuracy: 0.8556
Epoch 5/10
55000/55000 [==============================] - 8s 140us/sample - loss: 0.3959 - accuracy: 0.8603 - val_loss: 0.3760 - val_accuracy: 0.8728
Epoch 6/10
55000/55000 [==============================] - 8s 139us/sample - loss: 0.3794 - accuracy: 0.8663 - val_loss: 0.3934 - val_accuracy: 0.8536
Epoch 7/10
55000/55000 [===========================

## Time

I'm getting roughly .87 accuracy after 10 epochs, 8s per epoch for a training time of 84s.  Not bad. 

To time the whole output we can use the `time` library to mark the beginning and the end of our run. The `time.time()` command returns the number of seconds after January 1, 1970, 00:00:00 (UTC), the so called _epoch_. We can then measure the total time of a training run with 

    import time

    start = time.time()
    ## Run Training Code
    end = time.time()
    print(end - start)

## Activation Functions

Changing the activation functions using Keras is simple. In the creation of the dense layer, you just specify `activation=` and `kernel_initializer=`

    keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")
    
For the full list of activations, see https://keras.io/activations/. In our case, let's change all of our layers to ELU units with He initialization. The default kernel initializer is `glorot_uniform`, or uniformally distributed weights according to the Glorot normalization. If we change the activation we should also change the initializer. The list of initializers can be found at https://keras.io/initializers/.

In [11]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_16 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [12]:
## Compile the model with loss: sparse_categorical_crossentropy, optimizer: sgg, and metric: accuracy. 
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd",
             metrics=["accuracy"])

import time
start = time.time()

history = model.fit(X_train,y_train, epochs=10, 
          validation_data=[X_valid,y_valid])

## Fit the model for 10 epochs, vailidating on the validation data. 

end = time.time()
print("Training Time:",end - start)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 9s 159us/sample - loss: 0.6415 - accuracy: 0.7811 - val_loss: 0.5002 - val_accuracy: 0.8254
Epoch 2/10
55000/55000 [==============================] - 8s 150us/sample - loss: 0.4759 - accuracy: 0.8309 - val_loss: 0.4381 - val_accuracy: 0.8542
Epoch 3/10
55000/55000 [==============================] - 8s 144us/sample - loss: 0.4382 - accuracy: 0.8452 - val_loss: 0.4184 - val_accuracy: 0.8570
Epoch 4/10
55000/55000 [==============================] - 8s 148us/sample - loss: 0.4153 - accuracy: 0.8537 - val_loss: 0.4127 - val_accuracy: 0.8574
Epoch 5/10
55000/55000 [==============================] - 7s 133us/sample - loss: 0.3994 - accuracy: 0.8581 - val_loss: 0.3966 - val_accuracy: 0.8636
Epoch 6/10
55000/55000 [==============================] - 8s 146us/sample - loss: 0.3866 - accuracy: 0.8623 - val_loss: 0.3784 - val_accuracy: 0.8682
Epoch 7/10
55000/55000 [===========================

In this case we're getting slightly better results with the original ReLU function, but try training longer and see what you find. We're already doing fairly well here so we might not see too many gains. 

## Batch Normalization

Recall that Batch Normalization adds a layer before each dense layer that normalizes the input data. In Keras implementing batch normalization works exactly as you might expect, by adding a `keras.layers.BatchNormalization()` before any sequential layer we want to normalize.

In [13]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation="softmax"))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_18 (Dense)             (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_19 (Dense)             (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_20 (Dense)             (None, 10)               

Notice that the number of parameters has gone up. This is because each batch normalization layer has a number of trainable parameters governing the trained center and shape. Lets take a quick look at a batch normalization layer. Using the `model.layers` list, we will access the variables from the first layer and take a look at the 3136 variables inside:

In [15]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

We see that there are two trainable parameters, the center `gamma` and the scale `beta` and two parameters computed from the batch, the `moving_mean` $\hat{\mu}$ and the `moving_variance` $\hat{\sigma}^2$. The last two parameters are the moving average of the batch means and variances, to be used at prediction time. There are a lot of little hyperparameters to play with when using batch normalization, but usually the defaults can be used. 

A final note: batch normalization tries to properly center the data so a bias term would be redundant. Deep layers be default contain a bias term, so you may want to turn if off using `use_bias=False`.

In [21]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal",use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation="softmax"))

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 784)               0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 784)               3136      
_________________________________________________________________
dense_31 (Dense)             (None, 300)               235200    
_________________________________________________________________
batch_normalization_14 (Batc (None, 300)               1200      
_________________________________________________________________
dense_32 (Dense)             (None, 100)               30000     
_________________________________________________________________
batch_normalization_15 (Batc (None, 100)               400       
_________________________________________________________________
dense_33 (Dense)             (None, 10)              

In [22]:
## Compile the model with loss: sparse_categorical_crossentropy, optimizer: sgg, and metric: accuracy. 
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd",
             metrics=["accuracy"])

import time
start = time.time()

history = model.fit(X_train,y_train, epochs=10, 
          validation_data=[X_valid,y_valid])

## Fit the model for 10 epochs, vailidating on the validation data. 

end = time.time()
print("Training Time:",end - start)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 14s 259us/sample - loss: 0.5551 - accuracy: 0.8046 - val_loss: 0.4026 - val_accuracy: 0.8636
Epoch 2/10
55000/55000 [==============================] - 12s 225us/sample - loss: 0.4234 - accuracy: 0.8491 - val_loss: 0.3760 - val_accuracy: 0.8718
Epoch 3/10
55000/55000 [==============================] - 13s 229us/sample - loss: 0.3865 - accuracy: 0.8623 - val_loss: 0.3542 - val_accuracy: 0.8722
Epoch 4/10
55000/55000 [==============================] - 13s 228us/sample - loss: 0.3608 - accuracy: 0.8711 - val_loss: 0.3465 - val_accuracy: 0.8772
Epoch 5/10
55000/55000 [==============================] - 12s 227us/sample - loss: 0.3471 - accuracy: 0.8744 - val_loss: 0.3350 - val_accuracy: 0.8782
Epoch 6/10
55000/55000 [==============================] - 13s 228us/sample - loss: 0.3316 - accuracy: 0.8805 - val_loss: 0.3273 - val_accuracy: 0.8804
Epoch 7/10
55000/55000 [=====================

It has also been argued that bias layers should be added before the activation layer, so that the data is properly centered and scaled before activation. To do this, remove the activation call from the dense layers and add a `keras.layers.Activation("elu")` layer after each batch normalization layer:

In [23]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(300, kernel_initializer="he_normal",use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(100, kernel_initializer="he_normal",use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_12 (Flatten)         (None, 784)               0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 784)               3136      
_________________________________________________________________
dense_34 (Dense)             (None, 300)               235200    
_________________________________________________________________
batch_normalization_17 (Batc (None, 300)               1200      
_________________________________________________________________
activation (Activation)      (None, 300)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 100)               30000     
_________________________________________________________________
batch_normalization_18 (Batc (None, 100)             

In [24]:
## Compile the model with loss: sparse_categorical_crossentropy, optimizer: sgg, and metric: accuracy. 
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd",
             metrics=["accuracy"])

import time
start = time.time()

history = model.fit(X_train,y_train, epochs=10, 
          validation_data=[X_valid,y_valid])

## Fit the model for 10 epochs, vailidating on the validation data. 

end = time.time()
print("Training Time:",end - start)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 15s 266us/sample - loss: 0.5745 - accuracy: 0.8019 - val_loss: 0.4278 - val_accuracy: 0.8562
Epoch 2/10
55000/55000 [==============================] - 12s 226us/sample - loss: 0.4409 - accuracy: 0.8455 - val_loss: 0.3936 - val_accuracy: 0.8654
Epoch 3/10
55000/55000 [==============================] - 13s 229us/sample - loss: 0.4078 - accuracy: 0.8564 - val_loss: 0.3758 - val_accuracy: 0.8698
Epoch 4/10
55000/55000 [==============================] - 13s 237us/sample - loss: 0.3794 - accuracy: 0.8647 - val_loss: 0.3661 - val_accuracy: 0.8710
Epoch 5/10
55000/55000 [==============================] - 12s 225us/sample - loss: 0.3662 - accuracy: 0.8695 - val_loss: 0.3615 - val_accuracy: 0.8698
Epoch 6/10
55000/55000 [==============================] - 12s 220us/sample - loss: 0.3482 - accuracy: 0.8746 - val_loss: 0.3598 - val_accuracy: 0.8712
Epoch 7/10
55000/55000 [=====================

We see that our training is taking longer due to the extra steps added, but we are rewarded with a higher validation accuracy. 

## Gradient Clipping and Optimizers

Gradient clipping is done at the optimizer level by setting the `clipvalue` to your desired value. For example, to use gradient clipping with SGD we use

    optimizer = keras.optimizers.SGD(clipvalue=1.0)
    model.compile(loss="mse", optimizer=optimizer)
    
#### Momentum Optimization and Nesterov Accelerated Gradient

Momentum optimization can be similarly used by setting the `momentum = ` value in most optimizers to be nonzero. For example, to enable momentum optimization in SGD we use

    optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)
    
Similarly, to use the Nesterov trick we pass `nesterov = True` to the optimizer: 

    optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
    
#### RMSProp and Adam

RMSProp and Adam are build in keras optimizers and can be initialized using 

    optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)
 
and

    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    
For Adam, `beta_1` is the momentum falloff rate and `beta_2` is the scale falloff rate.

Nadam and AdaMax are also implemented: https://keras.io/optimizers/.  


### Implementing

To use a different optimizer, and to be able to specify it's parameters, we just change the compile line 

    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

to take an initialized optimizer:

    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

The code below implements the Adam optimizer.

In [16]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation="softmax"))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 784)               3136      
_________________________________________________________________
dense_21 (Dense)             (None, 300)               235500    
_________________________________________________________________
batch_normalization_4 (Batch (None, 300)               1200      
_________________________________________________________________
dense_22 (Dense)             (None, 100)               30100     
_________________________________________________________________
batch_normalization_5 (Batch (None, 100)               400       
_________________________________________________________________
dense_23 (Dense)             (None, 10)               

In [17]:
## Compile the model with loss: sparse_categorical_crossentropy, optimizer: Adam, and metric: accuracy. 

optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(loss="sparse_categorical_crossentropy",
             optimizer=optimizer,
             metrics=["accuracy"])

import time
start = time.time()

history = model.fit(X_train,y_train, epochs=10, 
          validation_data=[X_valid,y_valid])

## Fit the model for 10 epochs, vailidating on the validation data. 

end = time.time()
print("Training Time:",end - start)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 15s 271us/sample - loss: 0.4705 - accuracy: 0.8301 - val_loss: 0.3529 - val_accuracy: 0.8762
Epoch 2/10
55000/55000 [==============================] - 13s 239us/sample - loss: 0.3640 - accuracy: 0.8664 - val_loss: 0.3182 - val_accuracy: 0.8832
Epoch 3/10
55000/55000 [==============================] - 13s 244us/sample - loss: 0.3222 - accuracy: 0.8816 - val_loss: 0.3040 - val_accuracy: 0.8880
Epoch 4/10
55000/55000 [==============================] - 13s 236us/sample - loss: 0.2979 - accuracy: 0.8891 - val_loss: 0.2991 - val_accuracy: 0.8912
Epoch 5/10
55000/55000 [==============================] - 13s 229us/sample - loss: 0.2765 - accuracy: 0.8983 - val_loss: 0.2921 - val_accuracy: 0.8918
Epoch 6/10
55000/55000 [==============================] - 11s 206us/sample - loss: 0.2602 - accuracy: 0.9024 - val_loss: 0.2973 - val_accuracy: 0.8886
Epoch 7/10
55000/55000 [=====================

Just by changing the optimizer we're above 90% accuracy, and with no appreciable difference in training time.  

## Adding Regularization

Regularization is done at the layer level, allowing us to choose which layers we regularize if we with. To turn on regularization, pass a `kernal_regularizer = ` to the layer creation function

    layer = keras.layers.Dense(100, activation="elu",
        kernel_initializer="he_normal",
        kernel_regularizer=keras.regularizers.l2(0.01))
        
For $\ell_1$ of mixed $\ell_1-\ell_2$ regularization use `keras.regularizers.l1` or `keras.regularizers.l1_l2`. For Max-Norm regularization use `keras.constraints.max_norm(1.)`. The full list of regularizers can be found here: https://keras.io/regularizers/

In [25]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal",kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation="softmax",kernel_regularizer=keras.regularizers.l2(0.01)))

model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_13 (Flatten)         (None, 784)               0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 784)               3136      
_________________________________________________________________
dense_37 (Dense)             (None, 300)               235500    
_________________________________________________________________
batch_normalization_20 (Batc (None, 300)               1200      
_________________________________________________________________
dense_38 (Dense)             (None, 100)               30100     
_________________________________________________________________
batch_normalization_21 (Batc (None, 100)               400       
_________________________________________________________________
dense_39 (Dense)             (None, 10)              

In [26]:
## Compile the model with loss: sparse_categorical_crossentropy, optimizer: Adam, and metric: accuracy. 

optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(loss="sparse_categorical_crossentropy",
             optimizer=optimizer,
             metrics=["accuracy"])

import time
start = time.time()

history = model.fit(X_train,y_train, epochs=10, 
          validation_data=[X_valid,y_valid])

## Fit the model for 10 epochs, vailidating on the validation data. 

end = time.time()
print("Training Time:",end - start)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 17s 309us/sample - loss: 1.4729 - accuracy: 0.7901 - val_loss: 0.7201 - val_accuracy: 0.8258
Epoch 2/10
55000/55000 [==============================] - 13s 237us/sample - loss: 0.7462 - accuracy: 0.8016 - val_loss: 0.6652 - val_accuracy: 0.8346
Epoch 3/10
55000/55000 [==============================] - 12s 224us/sample - loss: 0.6774 - accuracy: 0.8170 - val_loss: 0.5854 - val_accuracy: 0.8522
Epoch 4/10
55000/55000 [==============================] - 13s 234us/sample - loss: 0.6243 - accuracy: 0.8261 - val_loss: 0.5939 - val_accuracy: 0.8350
Epoch 5/10
55000/55000 [==============================] - 12s 216us/sample - loss: 0.5938 - accuracy: 0.8329 - val_loss: 0.5447 - val_accuracy: 0.8528
Epoch 6/10
55000/55000 [==============================] - 12s 218us/sample - loss: 0.5737 - accuracy: 0.8363 - val_loss: 0.5559 - val_accuracy: 0.8404
Epoch 7/10
55000/55000 [=====================

Regularization is slowing down the training which is perhaps unsurprising , but it seems have really punished us for no particular gain in accuracy. We may be regularizing too much, but it could also be that while it takes longer to get where its going, the regularized classifier is more accurate on new data.

## Dropout

To add dropout to the layers of the network, we simply need to add a dropout layer before any dense layers:

    keras.layers.Dropout(rate=0.2)
    
Dropout tends to significantly slow convergence, but the results are much better if you have the training time.

In [27]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal",kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(10, activation="softmax",kernel_regularizer=keras.regularizers.l2(0.01)))

model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_14 (Flatten)         (None, 784)               0         
_________________________________________________________________
dropout (Dropout)            (None, 784)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 300)               235500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 10)              

In [28]:
## Compile the model with loss: sparse_categorical_crossentropy, optimizer: Adam, and metric: accuracy. 

optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(loss="sparse_categorical_crossentropy",
             optimizer=optimizer,
             metrics=["accuracy"])

import time
start = time.time()

history = model.fit(X_train,y_train, epochs=10, 
          validation_data=[X_valid,y_valid])

## Fit the model for 10 epochs, vailidating on the validation data. 

end = time.time()
print("Training Time:",end - start)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 6s 114us/sample - loss: 1.6822 - accuracy: 0.7522 - val_loss: 0.9546 - val_accuracy: 0.7970
Epoch 2/10
55000/55000 [==============================] - 6s 113us/sample - loss: 1.0104 - accuracy: 0.7601 - val_loss: 0.9187 - val_accuracy: 0.8046
Epoch 3/10
55000/55000 [==============================] - 6s 110us/sample - loss: 0.9931 - accuracy: 0.7629 - val_loss: 0.9213 - val_accuracy: 0.7918
Epoch 4/10
55000/55000 [==============================] - 8s 146us/sample - loss: 0.9893 - accuracy: 0.7661 - val_loss: 0.9187 - val_accuracy: 0.7940
Epoch 5/10
55000/55000 [==============================] - 8s 145us/sample - loss: 0.9851 - accuracy: 0.7632 - val_loss: 0.9307 - val_accuracy: 0.7922
Epoch 6/10
55000/55000 [==============================] - 10s 177us/sample - loss: 0.9798 - accuracy: 0.7656 - val_loss: 0.9197 - val_accuracy: 0.7928
Epoch 7/10
55000/55000 [==========================

# Question:

What about using dropout and batch normalization together? Does the order matter? For example, we could use

Dense -> Batch Norm -> Activation -> Dropout -> Dense

as the original paper suggests, but recent test suggest this is actually not the optimal order. Perform some test to try to determine for the network above which is the proper ordering. 